# HLMA 408: Lois gaussiennes et visualisation interactive

***
> __Auteur__: Joseph Salmon <joseph.salmon@umontpellier.fr>

On va illustrer des lois classiques en donnant leur densité, et des tirages aléatoires de ses lois.
On observe ainsi l'impact du centrage (en: *centering*) et de la mise à l'échelle (en: *mise à l'échelle*) en partant d'un $X$ centré-réduit, et en investiguant $Y=\frac{X-\mu}{\sigma}$.



Ce notebook est inspiré des posts suivants:
- https://medium.com/kapernikov/ipywidgets-with-matplotlib-93646718eb84
- https://medium.com/@jdchipox/how-to-interact-with-jupyter-33a98686f24e

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, laplace, t, f, expon, beta, cauchy
import ipywidgets  # ipywidgets>=7.5

In [2]:
%matplotlib widget

In [3]:
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "serif",
#     "font.serif": ["Palatino"]})

In [4]:
distributions_dict = {'Gaussienne': norm,
                      'Laplace': laplace,
                      'Exponentielle': expon,
                      'Cauchy': cauchy,
                      }
# plus d'exemples ici: https://docs.scipy.org/doc/scipy/reference/stats.html


## Visualisation : densités et tirages aléatoires

In [5]:
def make_box_layout():
    return ipywidgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px',
    )

In [6]:
class Sines(ipywidgets.HBox):

    def __init__(self):
        super().__init__()
        output = ipywidgets.Output()
        self.n_samples = 30
        self.xranges = (-5, 5)  # Bornes d'observation
        self.yranges = (0, 0.5)  # Bornes d'observation

        self.x = np.linspace(self.xranges[0], self.xranges[1], num=400)
        self.mu, self.sigma = 0, 1
        self.distribution = distributions_dict['Gaussienne']
        self.samples = self.distribution.rvs(
            size=self.n_samples, loc=self.mu, scale=self.sigma)
        self.size = 5
        self.initial_color = '#1a60e1'
        self.jitter = 0.10
        self.params = dict(
            #                            s=self.size,
            color=self.initial_color,
            alpha=0.50,
            linewidth=0.2,
            edgecolor="black"
        )

        with output:
            self.fig, self.ax = plt.subplots(2, 1, sharex=True,
                                             num='Densité et tirages aléatoires',
                                             constrained_layout=True, figsize=(6, 4))

        self.line, = self.ax[0].plot(self.x,
                                     self.distribution.pdf(
                                         self.x, loc=self.mu, scale=self.sigma),
                                     self.initial_color)

        self.ax[0].set_xlabel(r'Densité de $Y=\sigma \cdot X + \mu $')
        self.ax[0].set_xlim(self.xranges)
        self.ax[0].set_ylim(self.yranges)

        self.ax[1].set_xlabel(r'Tirages de $Y_i= \sigma \cdot X_i + \mu, i=1,\dots,n$')
        self.ax[1].set_xlim(self.xranges)
        self.ax[1].axes.get_yaxis().set_visible(False)
        self.ax[1].set_ylim(-2, 2)

        self.y = self.jitter * np.random.randn(self.n_samples,)

        self.points = self.ax[1].scatter(self.samples, self.y, **self.params)
        self.fig.canvas.toolbar_position = 'bottom'

        # define widgets
        style = {'description_width': '100px'}
        layout = {'width': '300px'}

        n_samples_slider = ipywidgets.IntSlider(
            value=self.n_samples, min=1, max=500, step=1, description="Nb tirages", style=style, layout=layout)
        mu_slider = ipywidgets.FloatSlider(
            value=0, min=-2, max=2, step=0.1, description='$\mu$', style=style, layout=layout)
        sigma_slider = ipywidgets.FloatSlider(
            value=1, min=0.5, max=3, step=0.1, description='$\sigma$', style=style, layout=layout)
        color_picker = ipywidgets.ColorPicker(
            value=self.initial_color,
            description='Couleur', style=style, layout=layout)
        jitter_slider = ipywidgets.FloatSlider(
            value=self.jitter, min=0.005, max=0.3, step=0.005, description='Frémissement', style=style, layout=layout)
        text_distribution = ipywidgets.Dropdown(
            options=list(distributions_dict),
            value='Gaussienne',
            description='Distribution', style=style, layout=layout)
        int_xrange_slider = ipywidgets.FloatRangeSlider(
            value=self.xranges,
            min=-5, max=5, step=0.1,
            description="Zoom en x", style=style, layout=layout)
        int_yrange_slider = ipywidgets.FloatRangeSlider(
            value=(self.yranges),
            min=-1, max=3, step=0.1,
            description="Zoom en y", style=style, layout=layout)

        controls = ipywidgets.VBox([
            n_samples_slider,
            mu_slider,
            sigma_slider,
            text_distribution,
            color_picker,
            int_xrange_slider,
            int_yrange_slider,
            jitter_slider
        ])

        controls.layout = make_box_layout()
        out_box = ipywidgets.Box([output])
        output.layout = make_box_layout()

        # Observed stuff
        n_samples_slider.observe(self.update_n_samples, 'value')
        mu_slider.observe(self.update_mu, 'value')
        sigma_slider.observe(self.update_sigma, 'value')
        color_picker.observe(self.line_color, 'value')
        text_distribution.observe(self.update_text_distribution, 'value')
        int_xrange_slider.observe(self.update_xrange_slider, 'value')
        int_yrange_slider.observe(self.update_yrange_slider, 'value')
        jitter_slider.observe(self.update_jitter_slider, 'value')

        # Add to children
        self.children = [controls, output]

    def update_mu(self, change):
        """Evolution with the mu parameter."""
        self.mu = change.new
        self.samples = self.distribution.rvs(
            size=self.n_samples, loc=self.mu, scale=self.sigma)
        self.line.set_ydata(self.distribution.pdf(
            self.x, loc=self.mu, scale=self.sigma))
        self.y = self.jitter * np.random.randn(self.n_samples,)
        self.points.set_offsets(np.c_[[self.samples, self.y]].T)
        self.fig.canvas.draw()

    def update_sigma(self, change):
        """Evolution with the sigma parameter."""
        self.sigma = change.new
        self.samples = self.distribution.rvs(
            size=self.n_samples, loc=self.mu, scale=self.sigma)
        self.line.set_ydata(self.distribution.pdf(
            self.x, loc=self.mu, scale=self.sigma))
        y = self.jitter * np.random.randn(self.n_samples,)
        self.points.set_offsets(np.c_[[self.samples, self.y]].T)
        self.fig.canvas.draw()

    def update_n_samples(self, change):
        """Evolution with the n_samples parameter."""
        self.n_samples = change.new
        self.samples = self.distribution.rvs(
            size=self.n_samples, loc=self.mu, scale=self.sigma)
        self.y = self.jitter * np.random.randn(self.n_samples,)
        self.points.set_offsets(np.c_[[self.samples, self.y]].T)
        self.fig.canvas.draw()

    def update_jitter_slider(self, change):
        """Evolution with the jitter parameter."""
        jitter_old = self.jitter
        self.jitter = change.new
        self.y *= self.jitter / jitter_old
        self.points.set_offsets(np.c_[[self.samples, self.y]].T)
        self.fig.canvas.draw()

    def line_color(self, change):
        self.initial_color = change.new
        self.line.set_color(self.initial_color)
        self.params['color'] = self.initial_color
        self.y = self.jitter * np.random.randn(self.n_samples,)
        self.points.set_offsets(np.c_[[self.samples, self.y]].T)
        self.points.set_color(self.initial_color)
        self.fig.canvas.draw()

    def update_xrange_slider(self, change):
        self.xranges = change.new
        self.line.set_ydata(self.distribution.pdf(
            self.x, loc=self.mu, scale=self.sigma))
        self.ax[0].set_xlim(self.xranges)
        self.ax[1].set_xlim(self.xranges)
        self.fig.canvas.draw()

    def update_yrange_slider(self, change):
        self.yranges = change.new
        self.ax[0].set_ylim(self.yranges)
        self.fig.canvas.draw()

    def update_text_distribution(self, change):
        self.distribution = distributions_dict[change.new]
        self.line.set_ydata(self.distribution.pdf(
            self.x, loc=self.mu, scale=self.sigma))
        self.samples = self.distribution.rvs(
            size=self.n_samples, loc=self.mu, scale=self.sigma)
        self.y = self.jitter * np.random.randn(self.n_samples,)
        self.points.set_offsets(np.c_[[self.samples, self.y]].T)
        self.points.set_color(self.initial_color)
        self.fig.canvas.draw()


Sines()

Sines(children=(VBox(children=(IntSlider(value=30, description='Nb tirages', layout=Layout(width='300px'), max…